In [6]:
%%capture
!pip install transformers==4.2.1
!pip install tensorflow-gpu==2.3.0 
print('Done!')

In [7]:
!pip install spacy==3.0

     |████████████████████████████████| 12.7MB 247kB/s 
     |████████████████████████████████| 1.1MB 31.7MB/s 
     |████████████████████████████████| 9.1MB 25.5MB/s 
     |████████████████████████████████| 460kB 33.0MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 92kB 10.5MB/s 
     |████████████████████████████████| 122kB 42.5MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=833ab09b04b4c9976abbeb96e9bf100bcbcc1dd5d6226905c7aabbcab5b6d016
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.

In [8]:
!python -m spacy download es_core_news_md

2021-05-20 18:12:14.311752: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
     |████████████████████████████████| 44.0MB 68kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [3]:
%reset
import pandas as pd
import tensorflow as tf

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
train_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/haha_2021_train.csv", sep=',')
train_dataframe[train_dataframe.is_humor==1]

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,humor_rating,humor_mechanism,humor_target
0,tweet1,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1,1,2,2,0,0,0,1.5000,NaN,NaN
1,tweet2,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,1,3,0,1,0,0,1.5000,NaN,NaN
2,tweet3,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,0,2,1,0,1,1,2.6000,NaN,NaN
6,tweet7,"—Buenas don Pepe, ¿me vende un litro de leche?...",1,1,1,1,1,1,0,2.5000,NaN,NaN
12,tweet13,#20CosasQueHacerAntesDeMorir: Enseñarles la di...,1,0,1,1,2,1,0,2.6000,reference,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
23991,tweet23992,¿Cuál es el dominio mas erótico?... www.pet.es...,1,2,2,1,0,0,0,1.3333,NaN,NaN
23993,tweet23994,"—¿Nivel de inglés? —100% —Traduzca ""Escoger un...",1,0,2,4,0,0,0,1.6667,wordplay,NaN
23994,tweet23995,"Era tan, pero tan mal jugador, que hizo un gol...",1,1,0,2,0,2,0,3.0000,NaN,NaN
23995,tweet23996,"—Estoy afuera de tu casa, sal. —Ya voy, pimienta.",1,0,2,1,1,1,0,2.2000,NaN,NaN


In [ ]:
mechanisms = train_dataframe[["text", "is_humor", "humor_mechanism"]]

In [ ]:
mechanisms = mechanisms[mechanisms.is_humor==1].dropna()

In [ ]:
mechanisms

,text,is_humor,humor_mechanism
12,#20CosasQueHacerAntesDeMorir: Enseñarles la di...,1,reference
20,Te quiero pero #YoTan Twitter y tú tan Facebook.,1,analogy
21,"-El alcohol es un veneno. –Si es un veneno, ¿P...",1,reference
22,Cambié mi contraseña de Twitter por ''incorrec...,1,irony
28,WhatsApp cayó varias veces en 2015 y vos todav...,1,insults
...,...,...,...
23973,"Doctor, no levanto la cabeza, no hablo con la ...",1,misunderstanding
23983,Que me parta un rayo (Un Siamés) #CHISTES,1,irony
23984,Cualquier cosa que hagas da siempre el 100%\n\...,1,wordplay
23988,"Que rápido se pasó el año, ni tiempo de adelga...",1,reference


In [ ]:
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import numpy as np

In [ ]:
tar = LabelEncoder().fit_transform(mechanisms["humor_mechanism"])

In [ ]:
tar

array([ 8,  1,  8, ..., 11,  8, 11])

In [ ]:
# Hay que ponderar categorías, aunque me da más miedo el tema de la tokenizacion
categories = sorted(np.unique(tar), reverse=False)
hist= Counter(tar) 
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()

In [ ]:
from transformers import RobertaConfig, TFRobertaModel, RobertaTokenizer

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 28.2MB/s 


In [ ]:
roberta = TFRobertaModel.from_pretrained("cardiffnlp/twitter-roberta-base")

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")

In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='cardiffnlp/twitter-roberta-base', vocab_size=50265, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [ ]:
roberta.trainable = False
roberta.summary()

Model: "tf_roberta_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
roberta (TFRobertaMainLayer) multiple                  124645632 
Total params: 124,645,632
Trainable params: 0
Non-trainable params: 124,645,632
_________________________________________________________________


In [ ]:
def get_model_inputs(tokenizer, data):
    encodings = tokenizer(data, truncation=True, padding='max_length', max_length=256, return_tensors="tf")
    inputs = {'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask']
            }
    return inputs

In [ ]:
get_model_inputs(tokenizer, train_dataframe["text"].tolist())

{'attention_mask': <tf.Tensor: shape=(24000, 256), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_ids': <tf.Tensor: shape=(24000, 256), dtype=int32, numpy=
 array([[   0,  487, 2088, ...,    1,    1,    1],
        [   0,  578,  846, ...,    1,    1,    1],
        [   0,   12, 5505, ...,    1,    1,    1],
        ...,
        [   0,  104, 2160, ...,    1,    1,    1],
        [   0, 7083, 5521, ...,    1,    1,    1],
        [   0, 5408, 4091, ...,    1,    1,    1]], dtype=int32)>}

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Flatten, ReLU, Dropout
from tensorflow.keras.models import Model

In [ ]:
from transformers import XLMRobertaTokenizer

In [ ]:
in_roberta = Input(shape=(256,), dtype="int64")
in_roberta_mask = Input(shape=(256,), dtype="int64")

roberta.trainable = False

roberta_out = roberta(in_roberta, in_roberta_mask)
roberta_out = roberta_out.last_hidden_state

x = Dense(256)(roberta_out)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.7)(x)


x = Dense(128)(roberta_out)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.7)(x)

x = Dense(32)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.7)(x)

x = Flatten()(x)

output = Dense(13, activation="softmax")(x)

model = Model([in_roberta, in_roberta_mask], output)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode TFBaseModelOutputWit 124645632   input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 256, 128)     98432       tf_roberta_model[3][0]

In [ ]:
inputs = get_model_inputs(tokenizer, train_dataframe["text"].tolist())

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
outputs = LabelEncoder().fit_transform(train_dataframe["humor_mechanism"].tolist())

In [ ]:
outputs = to_categorical(outputs)

In [ ]:
outputs.shape

(24000, 13)

In [ ]:
import tqdm

In [ ]:
epochs_max = 10
epochs_to_save = 5
batch_size = 32

for epoch in tqdm.tqdm(range(0, epochs_max, epochs_to_save)):
    print(f'Training model, epochs {epoch+1} - {epoch+epochs_to_save}')
    
    # entrenar el modelo. Opcionalmente, se puede suministrar datos de validación => validation_data=(val_inputs,val_blabels_t )
    model.fit([inputs["input_ids"], inputs["attention_mask"]] , y=outputs, epochs=epochs_to_save, batch_size=batch_size, validation_split = 0.2)




  0%|          | 0/2 [00:00<?, ?it/s]The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Training model, epochs 1 - 5
Epoch 1/5


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


600/600 [==============================] - ETA: 0s - loss: 1.0701 - accuracy: 0.8145

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


600/600 [==============================] - 447s 745ms/step - loss: 1.0701 - accuracy: 0.8145 - val_loss: 1.3204 - val_accuracy: 0.7071
Epoch 2/5
600/600 [==============================] - 450s 751ms/step - loss: 0.9205 - accuracy: 0.8232 - val_loss: 1.4177 - val_accuracy: 0.7071
Epoch 3/5
600/600 [==============================] - 450s 751ms/step - loss: 0.9120 - accuracy: 0.8232 - val_loss: 1.3146 - val_accuracy: 0.7071
Epoch 4/5
600/600 [==============================] - 450s 751ms/step - loss: 0.9123 - accuracy: 0.8232 - val_loss: 1.4732 - val_accuracy: 0.7071
Epoch 5/5
149/600 [======>.......................] - ETA: 4:39 - loss: 0.9265 - accuracy: 0.8190

KeyboardInterrupt: ignored

---
## Uso de Spacy para sacar cosas interesantes

In [19]:
import spacy
import es_core_news_md
import numpy as np
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import seaborn as sns
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import numpy as np

In [22]:
mechanisms.dropna()

,text,is_humor,humor_target
21,"-El alcohol es un veneno. –Si es un veneno, ¿P...",1,substance use
39,"–¿Tiene pastillas para la diarrea? –No. –Ok, d...",1,health
63,4 etapas de la vida: 1- Creer en Santa Claus. ...,1,age
70,-¡Policía! Dos chicas se están peleando por mí...,1,body shaming; women
73,Teníamos una farmacia pero la cerramos porque ...,1,professions
...,...,...,...
23899,—Oye Peña Nieto ¿Quién crees que sea el suceso...,1,professions
23932,QUE LE DIJO EL BORRACHO AL RECIEN NACIDO : BEB...,1,substance use
23949,¿Ponerme triste porque te caes? Jajaja! #NiQue...,1,technology
23967,La ex novia de Jaimito se quería lanzar desde ...,1,women


In [23]:
mechanisms = train_dataframe[["text", "is_humor", "humor_target"]].dropna()
tar = LabelEncoder().fit_transform(mechanisms["humor_target"].tolist())
categories = sorted(np.unique(tar), reverse=False)
hist= Counter(tar) 
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in categories]))
fig.show()

In [ ]:
mechanisms["text"][23932]

'QUE LE DIJO EL BORRACHO AL RECIEN NACIDO : BEBE \n#chistes #humor'

In [ ]:
test = mechanisms["text"][23932]

In [ ]:
nlp = es_core_news_md.load()

In [ ]:
doc_analysis = nlp(test)

In [ ]:
dataframe = pd.DataFrame({"Palabra": [], "Lema": [], "Categoria_gramatical": []})

for token in doc_analysis:
    dataframe = dataframe.append({"Palabra": token.text, "Lema": token.lemma_, "Categoria_gramatical": token.pos_}, ignore_index=True)
  
dataframe

,Palabra,Lema,Categoria_gramatical
0,QUE,que,SCONJ
1,LE,él,PRON
2,DIJO,DIJO,PROPN
3,EL,el,DET
4,BORRACHO,BORRACHO,PROPN
5,AL,AL,ADP
6,RECIEN,recien,NOUN
7,NACIDO,nacido,ADJ
8,:,:,PUNCT
9,BEBE,bebe,NOUN


Noun-based approach

In [ ]:
XSamples = []
for sample in mechanisms["text"]:
  doc_analysis = nlp(sample)
  sample = ""
  for token in doc_analysis:
    if token.pos_ == "NOUN" or token.pos_ == "ADJ":
      sample += token.lemma_ + " "
  
  if sample == '':
    sample += "nada"
  XSamples.append(sample)

XSamples

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SpanishStemmer

In [ ]:
def spanish_stemmer(sentence):
    stemmer = SpanishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word',
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))

In [ ]:
vectorizer = CountVectorizer(analyzer=spanish_stemmer)
X = vectorizer.fit_transform(XSamples)

In [ ]:
X.shape

(1629, 2225)

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [ ]:
target = LabelEncoder().fit_transform(mechanisms["humor_target"])
Y = to_categorical(target)

In [ ]:
np.unique(target)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52])

In [ ]:
Y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Definir una red sencilla, mucho ojo porque hay pocos datos y las categorías están superdesbalanceadas

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1Score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
input = Input(2225,)
x = Dense(128, activation="relu")(input)
x = Dropout(0.25)(x)
x = Dense(64, activation="sigmoid")(x)
x = Dropout(0.25)(x) 
x = BatchNormalization()(x)
output = Dense(53, activation="softmax")(x)

model = Model([input], output)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=['accuracy', f1Score])
model.summary()

Model: "functional_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 2225)]            0         
_________________________________________________________________
dense_98 (Dense)             (None, 128)               284928    
_________________________________________________________________
dropout_51 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_99 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_52 (Dropout)         (None, 64)                0         
_________________________________________________________________
batch_normalization_48 (Batc (None, 64)                256       
_________________________________________________________________
dense_100 (Dense)            (None, 53)              

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X.toarray(), Y, test_size=0.2)

In [ ]:
y_train_classes = np.argmax(Y, axis=1)
 
pesos = class_weight.compute_class_weight('balanced',
                                          np.unique(y_train_classes),
                                          y_train_classes)
pesos = dict(enumerate(pesos))  

In [ ]:
X_train.shape

(1303, 2225)

In [ ]:
Y_train.shape

(1303, 53)

In [ ]:
X_test.shape

(326, 2225)

In [ ]:
Y_test.shape

(326, 53)

In [ ]:
model.fit(X_train, Y_train, batch_size=16, validation_data=(X_test, Y_test), class_weight=pesos, epochs=50)

In [ ]:
from transformers import pipeline

NameError: ignored